# Aprendizaje Reforzado (Reinforcement Learing)

## Notas: 
1. Funciona como aprendiendo por prueba y error en una tarea especifica, necesitan un agente aleatorio para estos 'intentos'.
2. Existe un Agente(Agent) y un Entorno(Enviroment) del cual existe un proceso de accion entre el agente al entrono, obteniedno asi recompensas y/o observaciones

# Dependencias

In [1]:
!pip install stable_baselines3[extra]
!pip install gym[all]

  Using cached lz4-4.0.1-cp37-cp37m-win_amd64.whl (98 kB)
  Using cached mujoco-py-1.50.1.68.tar.gz (120 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached box2d-py-2.3.5.tar.gz (374 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached Cython-0.29.30-py2.py3-none-any.whl (985 kB)
  Using cached imageio-2.19.3-py3-none-any.whl (3.4 MB)
  Running setup.py clean for box2d-py
  Running setup.py clean for mujoco-py
Failed to build box2d-py mujoco-py
  Running setup.py install for box2d-py: started
  Running setup.py install for box2d-py: finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [16 lines of output]
  Using setuptools (version 47.1.0).
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.7
  creating build\lib.win-amd64-3.7\Box2D
  copying library\Box2D\Box2D.py -> build\lib.win-amd64-3.7\Box2D
  copying library\Box2D\__init__.py -> build\lib.win-amd64-3.7\Box2D
  creating build\lib.win-amd64-3.7\Box2D\b2
  copying library\Box2D\b2\__init__.py -> build\lib.win-amd64-3.7\Box2D\b2
  running build_ext
  building 'Box2D._Box2D' extension
  swigging Box2D\Box2D.i to Box2D\Box2D_wrap.cpp
  swig.exe -python -c++ -IBox2D -small -O -includeall -ignoremissing -w201 -globals b2Globals -outdir library\Box2D -keyword -w511 -D_SWIG_KWARGS -o Box2D\Box2D_wrap.cpp Box2D\Box2D.i
  error: command 'swig.exe' failed: No such file or directory
  [end of output]
  
  note: This error originates from a subprocess, 

In [2]:
import os
import gym
from stable_baselines3 import PPO #Algoritmo PPO, existen mas como: A2C,DDPG,DQN,HER,SAC,TD3
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [3]:
enviroment_name = 'CartPole-v0'
env = gym.make(enviroment_name)

In [16]:
env.action_space.sample()

1

In [17]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode {} , Score:{}'.format(episode,score))
env.close()

Episode 1 , Score:25.0
Episode 2 , Score:20.0
Episode 3 , Score:14.0
Episode 4 , Score:33.0
Episode 5 , Score:19.0
Episode 6 , Score:21.0
Episode 7 , Score:16.0
Episode 8 , Score:26.0
Episode 9 , Score:14.0
Episode 10 , Score:61.0


In [18]:
#Espacio de Observacion: Posicion, Velocidad, Angulo, Velocidad Angular
#Espacio de Accion: Mover Izquierda, Mover Derecha
env.observation_space.sample()
# RT - env.observation_space

array([ 2.5418360e+00, -8.7200955e+37,  9.6643671e-02,  2.9022524e+38],
      dtype=float32)

In [19]:
log_path = os.path.join('Training','Logs')

In [20]:
env = gym.make(enviroment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log = log_path)

Using cpu device


In [21]:
model.learn(total_timesteps = 20000)

Logging to Training\Logs\PPO_5
-----------------------------
| time/              |      |
|    fps             | 807  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1064        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008763951 |
|    clip_fraction        | 0.0897      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -3.14e-05   |
|    learning_rate        | 0.0003      |
|    loss                 | 6.18        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0143     |
|    value_loss           | 46.2        |
-----------------------------------------
---

## Guardar y recargar

In [22]:
PPO_Path = os.path.join('Training','Saved Models','PPO_Model_Cartpole')

In [23]:
model.save(PPO_Path)

In [24]:
del model

In [25]:
model = PPO.load(PPO_Path, env = env)

## Evaluacion del modelo de RL

In [26]:
# si score =200 esta resuleto
evaluate_policy(model, env, n_eval_episodes = 5, render = True)

c:\users\julia\appdata\local\programs\python\python37\lib\site-packages\stable_baselines3\common\evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


(200.0, 0.0)

In [27]:
# SCORE Y DESVIACION ESTANDAR
env.close()

In [28]:
episodes = 10
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action , _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode {} , Score:{}'.format(episode,score))
env.close()

Episode 1 , Score:[200.]
Episode 2 , Score:[200.]
Episode 3 , Score:[176.]
Episode 4 , Score:[200.]
Episode 5 , Score:[200.]
Episode 6 , Score:[155.]
Episode 7 , Score:[150.]
Episode 8 , Score:[200.]
Episode 9 , Score:[120.]
Episode 10 , Score:[200.]
